## Installing and importing libraries

In [1]:
#%pip install huggingface_hub - q
#%pip install git+https://github.com/facebookresearch/segment-anything-2/ -q
#%pip install -r https://raw.githubusercontent.com/facebookresearch/segment-anything-2/main/requirements.txt - q
%pip install plotly -q
%pip install pydicom scikit-image albumentations rasterio -q


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
# System, file and general.
import sys
import glob
from glob import glob
import random
import os
import shutil
from tqdm import tqdm
import math
import time
import copy
import joblib
from collections import defaultdict
import gc
from IPython import display as ipd

# Data manipulation and analysis.
import pandas as pd
import numpy as np
pd.options.plotting.backend = "plotly"
tqdm.pandas()
import pydicom as dicom
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle

# Image.
import cv2
from PIL import Image, ImageOps
import albumentations as A
import imageio
from skimage.transform import resize
from skimage.io import imsave
from skimage import io 
from sklearn.model_selection import StratifiedKFold, KFold, StratifiedGroupKFold
from skimage.measure import regionprops
from scipy.ndimage import label

# Torch.
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torch.optim as optim
from torch.utils.data import Dataset
from torchvision import transforms
from torch.utils.data import DataLoader
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch.optim import lr_scheduler
from torch.utils.data import Dataset, DataLoader
from torch.cuda import amp

/Users/jakecordery/Desktop/dissertation-york/.venv/lib/python3.12/site-packages/albumentations/__init__.py:24: UserWarning: A new version of Albumentations is available: 1.4.21 (you have 1.4.20). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()


## Notebook Structure:
### Load in data
- Load in csv files for default and 'improved' data
- Combine the default and additional dataframes
- Add filenames to the dataframes
    
### Preprocess imgs
- Load in dicom stacks
- Crop nessecary imgs
- Save all as png
    
### 

## Load and clean data

In [3]:
train = pd.read_csv("rsna-2024-lumbar-spine-degenerative-classification/train.csv")
train_label_coordinates = pd.read_csv("rsna-2024-lumbar-spine-degenerative-classification/train_label_coordinates.csv")
train_series_descriptions = pd.read_csv("rsna-2024-lumbar-spine-degenerative-classification/train_series_descriptions.csv")
test_series_description  = pd.read_csv("rsna-2024-lumbar-spine-degenerative-classification/test_series_descriptions.csv")
coords = pd.read_csv("coords_rsna_improved.csv")

train_labels = pd.merge(train_series_descriptions, train_label_coordinates, on = ['study_id', 'series_id'])

# Align formatting with the train.csv file as well as the sample submission.
condition = train_labels['condition'].to_numpy()
reformatted_condition = np.array([cond.lower().replace(' ', '_') for cond in condition])
train_labels.iloc[::, 4] = reformatted_condition

level = train_labels['level'].to_numpy()
reformatted_level = np.array([l.lower().replace('/', '_') for l in level])
train_labels.iloc[::, 5] = reformatted_level

series_description = train_labels['series_description'].to_numpy()
reformatted_sd = np.array([d.replace(' ', '_').replace('/', '_') for d in series_description])
train_labels.iloc[::, 2] = reformatted_sd

def is_nan(value):
    return pd.isna(value)

def clean_nan_values(data):
    for index, row in data.iterrows():
        for i in range(len(row)):
            if is_nan(row[i]):
                row[i] = "Normal/Mild"
    return data

train = clean_nan_values(train)
train_labels

/var/folders/j9/bcpct8w966j66c8j5tg5wd4c0000gn/T/ipykernel_39353/2959692717.py:28: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if is_nan(row[i]):
/var/folders/j9/bcpct8w966j66c8j5tg5wd4c0000gn/T/ipykernel_39353/2959692717.py:29: FutureWarning: Series.__setitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To set a value by position, use `ser.iloc[pos] = value`
  row[i] = "Normal/Mild"


,study_id,series_id,series_description,instance_number,condition,level,x,y
0,4003253,702807833,Sagittal_T2_STIR,8,spinal_canal_stenosis,l1_l2,322.831858,227.964602
1,4003253,702807833,Sagittal_T2_STIR,8,spinal_canal_stenosis,l2_l3,320.571429,295.714286
2,4003253,702807833,Sagittal_T2_STIR,8,spinal_canal_stenosis,l3_l4,323.030303,371.818182
3,4003253,702807833,Sagittal_T2_STIR,8,spinal_canal_stenosis,l4_l5,335.292035,427.327434
4,4003253,702807833,Sagittal_T2_STIR,8,spinal_canal_stenosis,l5_s1,353.415929,483.964602
...,...,...,...,...,...,...,...,...
48687,4290709089,4237840455,Sagittal_T1,11,left_neural_foraminal_narrowing,l1_l2,219.465940,97.831063
48688,4290709089,4237840455,Sagittal_T1,12,left_neural_foraminal_narrowing,l2_l3,205.340599,140.207084
48689,4290709089,4237840455,Sagittal_T1,12,left_neural_foraminal_narrowing,l3_l4,202.724796,181.013624
48690,4290709089,4237840455,Sagittal_T1,12,left_neural_foraminal_narrowing,l4_l5,202.933333,219.733333


In [4]:
spinal_canal_columns = train.filter(like='spinal_canal').columns
train[spinal_canal_columns] = train[spinal_canal_columns].fillna('Normal/Mild')

def fill_nan_left_and_right(df, column):
    target_columns = df.filter(like=column).columns 
    null_rows = df[df[target_columns].isnull().any(axis=1)]
    for level in ['l1_l2', 'l2_l3', 'l3_l4', 'l4_l5', 'l5_s1']:
        left_col = f"left_{column}_{level}"
        right_col = f"right_{column}_{level}"
        for index, row in null_rows.iterrows():
            if pd.isna(row[left_col]) and pd.isna(row[right_col]):
                df.at[index, left_col] = 'Normal/Mild'
                df.at[index, right_col] = 'Normal/Mild'
            
            elif pd.isna(row[right_col]):
                df.at[index, right_col] = row[left_col]
            
            elif pd.isna(row[left_col]):
                df.at[index, left_col] = row[right_col]
            else:
                continue
    return df

train = fill_nan_left_and_right(train, 'neural_foraminal_narrowing')
train = fill_nan_left_and_right(train, 'subarticular_stenosis')

In [5]:
coords = pd.read_csv('coords_rsna_improved.csv')

coords = coords.drop(['side'], axis = 1)
coords = coords.drop(coords.columns[0], axis=1)
coords['level'] = [level.lower().replace('/', '_') for level in coords['level'].to_list()]

coords = coords[['study_id', 'series_id', 'instance_number', 'condition', 'level', 'relative_x', 'relative_y']]
coords

,study_id,series_id,instance_number,condition,level,relative_x,relative_y
0,3996069892,10996,13,Left Neural Foraminal Narrowing,l1_l2,0.521148,0.325282
1,3996069892,10996,6,Right Neural Foraminal Narrowing,l1_l2,0.516856,0.319701
2,3996069892,10996,12,Left Neural Foraminal Narrowing,l2_l3,0.493773,0.400966
3,3996069892,10996,6,Right Neural Foraminal Narrowing,l2_l3,0.504032,0.385531
4,3996069892,10996,12,Left Neural Foraminal Narrowing,l3_l4,0.487331,0.475040
...,...,...,...,...,...,...,...
58730,916362094,4294540297,10,Spinal Canal Stenosis,l3_l4,0.511282,0.571751
58731,916362094,4294540297,-1,Spinal Canal Stenosis,l4_l5,0.308594,0.667969
58732,916362094,4294540297,9,Spinal Canal Stenosis,l4_l5,0.516697,0.671029
58733,916362094,4294540297,-1,Spinal Canal Stenosis,l5_s1,0.328125,0.796875


## Preprocess DICOM MRIs 

In [6]:
def load_dcm_img(dcm_path) -> np.ndarray:
    dcm = dicom.dcmread(dcm_path)
    img: np.ndarray = dcm.pixel_array
    img = img.clip(np.percentile(img, 1), np.percentile(img, 99))
    img = img - np.min(img)
    img = img / np.max(img)
    img = (img * 255).astype(np.uint8)
    return img

def convert_to_8bit(x):
    lower, upper = np.percentile(x, (1, 99))
    x = np.clip(x, lower, upper)
    x = x - np.min(x)
    x = x / np.max(x)
    return (x * 255).astype("uint8")

def load_dicom_stack(dicom_folder, plane, reverse_sort=False):
    dicom_files = glob(os.path.join(dicom_folder, "*.dcm"))
    if not dicom_files:
        raise ValueError(f"No DICOM files found in {dicom_folder}")
        
    dicoms = [dicom.dcmread(f) for f in dicom_files]
    plane_index = {"sagittal": 0, "coronal": 1, "axial": 2}[plane.lower()]
    positions = np.asarray([float(d.ImagePositionPatient[plane_index]) for d in dicoms])

    idx = np.argsort(-positions if reverse_sort else positions)
    dicoms = [dicoms[i] for i in idx]
    max_shape = max([d.pixel_array.shape for d in dicoms])

    # Normalize the shape by padding with zeros or resizing.
    normalized_arrays, padding_array = [], []
    for d in dicoms:
        img = d.pixel_array.astype("float32")
        if img.shape != max_shape:
            # Resize or pad the image to the max_shape.
            padded_img = np.zeros(max_shape, dtype="float32")
            padded_img[:img.shape[0], :img.shape[1]] = img
            normalized_arrays.append(padded_img)

            x_padding = max_shape[1] - img.shape[1]
            y_padding = max_shape[0] - img.shape[0]
            padding_array.append((x_padding // 2, y_padding // 2))
        else:
            normalized_arrays.append(img)
            padding_array.append((0, 0))

    array = np.stack(normalized_arrays)
    ipp = np.asarray([d.ImagePositionPatient for d in dicoms]).astype("float")[idx]

    return {
        "array": convert_to_8bit(array), 
        "positions": ipp, 
        "pixel_spacing": np.asarray(dicoms[0].PixelSpacing).astype("float"),
        "padding_array": padding_array
    }

In [7]:
TRAIN_IMG_DIR = "rsna-2024-lumbar-spine-degenerative-classification/train_images"
normal_tl = "normal_train_labels.csv"

if not os.path.exists(normal_tl):
    for study_folder_path in tqdm(os.listdir(TRAIN_IMG_DIR)):
        series_folders = os.listdir(os.path.join(TRAIN_IMG_DIR, study_folder_path))
        for idx, series_folder in enumerate(series_folders):
            dcm_folder_path = os.path.join(TRAIN_IMG_DIR, study_folder_path, series_folder)

            df = train_labels[train_labels["series_id"] == int(series_folder)]
            try:
                series_desc = df["series_description"].values[0]
            except:
                print(f"Series description for series_id: {series_folder} not found! Skipping.")
                continue

            plane = "sagittal" if series_desc != "Axial T2" else "axial"
            dicom_data_3d = load_dicom_stack(dcm_folder_path, plane = plane)
            padding_array = dicom_data_3d["padding_array"]
            
            for i in range(len(df["x"].to_list())):
                x_pad, y_pad = padding_array[i]
                df.iloc[i, df.columns.get_loc("x")] = df.iloc[i, df.columns.get_loc("x")] + x_pad
                df.iloc[i, df.columns.get_loc("y")] = df.iloc[i, df.columns.get_loc("y")] + y_pad


            pixel_spacing = dicom_data_3d["pixel_spacing"]
            img = dicom_data_3d["array"][idx]
            img_shape = img.shape

            x_scale_factor = 224 / img_shape[1]
            y_scale_factor = 224 / img_shape[0]

            train_labels.loc[df.index, "x"] = df["x"] * (x_scale_factor)
            train_labels.loc[df.index, "y"] = df["y"] * (y_scale_factor)
        

In [8]:
# Verify Coordinates are between 0 and 224.
print(len(train_labels))
tl_in_lims = train_labels[(train_labels["x"] >= 0) & (train_labels["x"] <= 224) & (train_labels["y"] >= 0) & (train_labels["y"] <= 224)]
print(len(tl_in_lims))

48692
18114


In [9]:
if os.path.exists(normal_tl):
    train_labels = pd.read_csv(normal_tl)
else:
    train_labels.to_csv(normal_tl, index=False)

In [10]:
def resample_img(img, pixel_spacing, new_spacing):
    current_spacing = np.array(pixel_spacing, dtype=np.float32)
    resize_factor = current_spacing / new_spacing
    new_shape = np.round(img.shape * resize_factor).astype(int)
    resampled_image = resize(img, new_shape, preserve_range=True, anti_aliasing=True)
    return resampled_image

def resize_img(img):
    return resize(img, (224, 224), preserve_range=True, anti_aliasing=True)

def normalize_img(img):
    return (img - np.min(img)) / (np.max(img) - np.min(img))

def preprocess_img(img, pixel_spacing):
    resampled_img = resample_img(img, pixel_spacing, (1.0, 1.0))
    resized_img = resize_img(resampled_img)
    #denoised_img = reduce_noise(resized_img)
    normalized_img = normalize_img(resized_img)
    return normalized_img

In [11]:
"""Functions which get the dicom info and use it to create a preprocessed img, whilst also scaling the coords correctly"""
def save_img(img, filename, save_folder):
    os.makedirs(save_folder, exist_ok = True)
    imageio.imwrite(os.path.join(save_folder, filename), (img * 255).astype(np.uint8))

In [12]:
if not os.path.exists("train_png"):
    for study_folder_path in tqdm(os.listdir(TRAIN_IMG_DIR)):
        series_folders = os.listdir(os.path.join(TRAIN_IMG_DIR, study_folder_path))
        for series_folder in series_folders:
            dcm_folder_path = os.path.join(TRAIN_IMG_DIR, study_folder_path, series_folder)
            df = train_series_descriptions[train_series_descriptions["series_id"] == int(series_folder)]
            try:
                series_desc = df["series_description"].values[0][0]
            except:
                print(f"Series description for series_id: {series_folder} not found! Skipping.")
                continue

            plane = "sagittal" if series_desc != "Axial T2" else "axial"
            dicom_data_3d = load_dicom_stack(dcm_folder_path, plane = plane)
            
            if dicom_data_3d["array"].shape[0] == 0:
                print(f"Can't loop over size zero dicom data: {dicom_data_3d["array"].shape}")
                continue
            
            for k in range(dicom_data_3d["array"].shape[0]):
                img = dicom_data_3d["array"][k]
                pixel_spacing = dicom_data_3d["pixel_spacing"]
                processed_img = preprocess_img(img, pixel_spacing)
                save_folder = f"train_png/{study_folder_path}/{series_folder}"
                filename = f"{k+1}.png"
                save_img(processed_img, filename=filename, save_folder=save_folder)
           

In [13]:
d = {"study_id": "first", "series_description": "first", "condition": "first", "level": list, "x": list, "y": list}
train_labels_grouped = train_labels.groupby(["series_id", "instance_number"], as_index=False).aggregate(d).reindex(columns=train_labels.columns)

In [14]:
def read_img(img_file):
    img_file = img_file.strip()
    #Get img_file path, issue of a '/x' on the end.
    if img_file[-1] == 'x':
        img_file = img_file.removesuffix('/x')
    return Image.open(img_file)

In [15]:
IMG_PATHS = glob(os.path.join("train_png", "*/*/*png"))

ST1_IMG_PATHS, ST2_IMG_PATHS, AX_IMG_PATHS = [], [], []
st1_train_df = train_series_descriptions[(train_series_descriptions["series_description"] == "Sagittal_T1") | (train_series_descriptions["series_description"] == "Sagittal T1")]
st1_sids = st1_train_df["series_id"].to_list()
st2_train_df = train_series_descriptions[(train_series_descriptions["series_description"] == "Sagittal_T2_STIR") | (train_series_descriptions["series_description"] == "Sagittal T2/STIR")]
st2_sids = st2_train_df["series_id"].to_list()
ax_train_df = train_series_descriptions[(train_series_descriptions["series_description"] == "Axial_T2") | (train_series_descriptions["series_description"] == "Axial T2")]
ax_sids = ax_train_df["series_id"].to_list()

for path in tqdm(IMG_PATHS, maxinterval=len(IMG_PATHS)):
    if any(str(sid) in path for sid in st1_sids):
        ST1_IMG_PATHS.append(path)
    elif any(str(sid) in path for sid in st2_sids):
        ST2_IMG_PATHS.append(path)
    elif any(str(sid) in path for sid in ax_sids):
        AX_IMG_PATHS.append(path)

print(len(IMG_PATHS), len(ST1_IMG_PATHS), len(ST2_IMG_PATHS), len(AX_IMG_PATHS))

100%|██████████| 147218/147218 [00:43<00:00, 3415.62it/s]

147218 33670 33569 79979


### Display some images.

In [70]:
# Choosing patient 4003253, as they have many conditions.
ex_df = train_labels[train_labels['study_id'] == 4003253]
print(ex_df)

def display_patient(df):
    df = df.astype(str)
    df[['x', 'y']] = df[['x', 'y']].astype(float)
    for i in df['instance_number'].to_list():
        temp_df = df[df['instance_number'] == i]
        filepath = os.path.join('train_png', str(temp_df['study_id'].unique()[0]), str(temp_df['series_id'].unique()[0]), f'{str(i)}.png')
        if os.path.exists(filepath):
            img = cv2.imread(filepath)
            plt.imshow(img)
            plt.scatter(temp_df['x'].to_numpy(), temp_df['y'].to_numpy(), c='red')
            #plt.title(f'Study {temp_df['study_id'].unique(), temp_df['series_id'].unique()}\n{temp_df['condition']}')
            plt.axis('off')
            plt.show()
        else:
            print(f"{filepath} is not a valid filepath!")

#display_patient(ex_df)

    study_id   series_id series_description  instance_number  \
0    4003253   702807833   Sagittal_T2_STIR                8   
1    4003253   702807833   Sagittal_T2_STIR                8   
2    4003253   702807833   Sagittal_T2_STIR                8   
3    4003253   702807833   Sagittal_T2_STIR                8   
4    4003253   702807833   Sagittal_T2_STIR                8   
5    4003253  1054713880        Sagittal_T1                4   
6    4003253  1054713880        Sagittal_T1                4   
7    4003253  1054713880        Sagittal_T1                5   
8    4003253  1054713880        Sagittal_T1                6   
9    4003253  1054713880        Sagittal_T1                6   
10   4003253  1054713880        Sagittal_T1               11   
11   4003253  1054713880        Sagittal_T1               11   
12   4003253  1054713880        Sagittal_T1               11   
13   4003253  1054713880        Sagittal_T1               12   
14   4003253  1054713880        Sagittal

## Segment images from spider dataset
### Inspired by Kaggle user Tabassum_Nova's notebook: https://www.kaggle.com/code/tabassumnova/segmentation-unet-inference-zenodo-spider?scriptVersionId=187625847

In [69]:
from pretrained_unet import *

fold  empty
0.0   False    341
      True     127
1.0   False    297
      True     132
Name: id, dtype: int64

/Users/jakecordery/Desktop/Kaggle Competiton/lumbarSpineDegeneration/pretrained_unet.py:203: UserWarning: Argument 'alpha_affine' is not valid and will be ignored.
  A.ElasticTransform(alpha=1, sigma=50, alpha_affine=None, p=1.0)


ModuleNotFoundError: No module named 'segmentation_models_pytorch'

In [ ]:
# Import pretrained model.


## Use the now trained U-Net to segment RSNA MRIs

In [ ]:
from lumbar_dataset import *
model_path = "best_epoch-00.bin"
model = load_model(model_path).to(CFG.device)
model.eval()

st2_predict_dataset = RSNATrainPredict(ST2_IMG_PATHS)
print(len(st2_predict_dataset))
segmentation_dataloader = DataLoader(st2_predict_dataset, batch_size=64,
                          num_workers=8, shuffle=False, pin_memory=True)

def save_prediction(img_path, pred):
    filename = img_path.replace('train_png', 'nnUNet_segments').replace('png', 'npy')
    folder_path = "/".join(filename.split('/')[:-1])
    os.makedirs(folder_path, exist_ok=True)
    gray_mask = pred[:, :, 0]
    np.save(filename, gray_mask)

def process_and_save(data):
    img, img_paths = data["img"].to(CFG.device), data["img_path"]
    with torch.no_grad():
        pred = model(img)
        pred = (nn.Sigmoid()(pred) > 0.5).float()
    pred = pred.detach().cpu().numpy()
    pred = np.transpose(pred, (0, 2, 3, 1)) 
    for k in range(pred.shape[0]): #Batch_size.
        current_pred, current_instance = pred[k], img_paths[k].split('/')[-1].split('.')[0]
        if int(current_instance) <= 3 and np.count_nonzero(current_pred == 1.) > 50: #Need to mess around with it to find the best miniumum number of pixels.
            save_prediction(img_paths[k], current_pred) 
        elif int(current_instance) <= 3 and np.count_nonzero(current_pred == 1.) > 50: #Again, do testing.
            save_prediction(img_paths[k], current_pred) 
        elif np.count_nonzero(current_pred == 1.) > 50:
            save_prediction(img_paths[k], current_pred) 

if not os.path.exists("nnUNet_segments"):
    for data in tqdm(segmentation_dataloader, total=int(len(st2_predict_dataset) / 64)):
        process_and_save(data)


NameError: name 'load_model' is not defined

In [ ]:
st1_predict_dataset = RSNATrainPredict(ST1_IMG_PATHS)
print(len(st1_predict_dataset))
segmentation_dataloader = DataLoader(st1_predict_dataset, batch_size=64,
                          num_workers=8, shuffle=False, pin_memory=True)

if not os.path.exists("nnUNet_segments"):
    for data in tqdm(segmentation_dataloader, total=int(len(st1_predict_dataset) / 64)):
        process_and_save(data)

print(len(glob(os.path.join("nnUNet_segments", "*/*/*npy"))))

NameError: name 'RSNATrainPredict' is not defined

## nnU-Net dataset setup and training
Isensee, F., Jaeger, P. F., Kohl, S. A., Petersen, J., & Maier-Hein, K. H. (2021). 
nnU-Net: a self-configuring method for deep learning-based biomedical image segmentation. 
Nature methods, 18(2), 203-211.

In [ ]:
UNET_ST1_DIR = "nnUNet_raw/Dataset001_ST1_Degeneration"
UNET_ST2_DIR = "nnUNet_raw/Dataset002_ST2_Degeneration"
os.makedirs(os.path.join(UNET_ST1_DIR, "imagesTr"), exist_ok = True)
os.makedirs(os.path.join(UNET_ST1_DIR, "imagesTs"), exist_ok = True)
os.makedirs(os.path.join(UNET_ST1_DIR, "labelsTr"), exist_ok = True)

os.makedirs(os.path.join(UNET_ST2_DIR, "imagesTr"), exist_ok = True)
os.makedirs(os.path.join(UNET_ST2_DIR, "imagesTs"), exist_ok = True)
os.makedirs(os.path.join(UNET_ST2_DIR, "labelsTr"), exist_ok = True)

os.makedirs("nnUNet_preprocessed", exist_ok = True)
os.makedirs("nnUNet_results", exist_ok = True)

In [30]:
"""Create dataset.json"""
import json
s1,s2 = "sagittal_t1", "sagittal_t2"
lf,rf,scs = "left_neural_foraminal_narrowing", "right_neural_foraminal_narrowing", "spinal_canal_stenosis"
l1,l2,l3,l4,l5 = "l1_l2", "l2_l3", "l3_l4", "l4_l5", "l5_s1"
n,m,s,i = "normal_mild", "moderate", "severe", "intact"

st1_classes = {
        "background": 0,
        "other_disc": 1,

        f"{lf}_{l1}_{n}": 2, #Sagittal T1: left_neural_foraminal_narrowing normal_mild/moderate/severe.
        f"{lf}_{l2}_{n}": 3,
        f"{lf}_{l3}_{n}": 4,
        f"{lf}_{l4}_{n}": 5,
        f"{lf}_{l5}_{n}": 6,
        f"{lf}_{l1}_{m}": 7,
        f"{lf}_{l2}_{m}": 8,
        f"{lf}_{l3}_{m}": 9,
        f"{lf}_{l4}_{m}": 10,
        f"{lf}_{l5}_{m}": 11,
        f"{lf}_{l1}_{s}": 12,
        f"{lf}_{l2}_{s}": 13,
        f"{lf}_{l3}_{s}": 14,
        f"{lf}_{l4}_{s}": 15,
        f"{lf}_{l5}_{s}": 16,

        f"{rf}_{l1}_{n}": 17, #Sagittal T1: right_neural_foraminal_narrowing normal_mild/moderate/severe.
        f"{rf}_{l2}_{n}": 18,
        f"{rf}_{l3}_{n}": 19,
        f"{rf}_{l4}_{n}": 20,
        f"{rf}_{l5}_{n}": 21,
        f"{rf}_{l1}_{m}": 22,
        f"{rf}_{l2}_{m}": 23,
        f"{rf}_{l3}_{m}": 24,
        f"{rf}_{l4}_{m}": 25,
        f"{rf}_{l5}_{m}": 26,
        f"{rf}_{l1}_{s}": 27,
        f"{rf}_{l2}_{s}": 28,
        f"{rf}_{l3}_{s}": 29,
        f"{rf}_{l4}_{s}": 30,
        f"{rf}_{l5}_{s}": 31,

        f"{s1}_l1": 32, #Sagittal T1 disc.
        f"{s1}_l2": 33,
        f"{s1}_l3": 34,
        f"{s1}_l4": 35,
        f"{s1}_l5": 36,
        f"{s1}_s1": 37,
    }

st2_classes = {
        "background": 0,
        "other_disc": 1,

        f"{scs}_{l1}_{n}": 2, #Sagittal T2/STIR spinal_canal_stenosis normal_mild/moderate/severe.
        f"{scs}_{l2}_{n}": 3,
        f"{scs}_{l3}_{n}": 4,
        f"{scs}_{l4}_{n}": 5,
        f"{scs}_{l5}_{n}": 6,
        f"{scs}_{l1}_{m}": 7,
        f"{scs}_{l2}_{m}": 8,
        f"{scs}_{l3}_{m}": 9,
        f"{scs}_{l4}_{m}": 10,
        f"{scs}_{l5}_{m}": 11,
        f"{scs}_{l1}_{s}": 12,
        f"{scs}_{l2}_{s}": 13,
        f"{scs}_{l3}_{s}": 14,
        f"{scs}_{l4}_{s}": 15,
        f"{scs}_{l5}_{s}": 16, 

        f"{s2}_l1": 17, #Sagittal T2/STIR disc.
        f"{s2}_l2": 18,
        f"{s2}_l3": 19,
        f"{s2}_l4": 20,
        f"{s2}_l5": 21,
        f"{s2}_s1": 22,
}

In [ ]:
"""Add 'inferenced' degenerative rows to the train_labels dataframe."""
def inference_rows(rows, instance):
    rows = rows.drop_duplicates(subset = ["level"])
    rows["instance_number"] = instance
    return rows

erows = pd.DataFrame(columns = train_labels.columns)
sagittal_idf = train_labels[(train_labels["condition"] != "left_subarticular_stenosis") | (train_labels["condition"] != "right_subarticular_stenosis")]
print(len(sagittal_idf))

for idx, chunk in sagittal_idf.groupby("series_id"):
    chunk = chunk.reset_index()
    if len(chunk) < 2:
        continue # Skip any with 1 row in as no data.

    for instance in np.unique(chunk["instance_number"].to_numpy()):
        valid_ins = [instance - 1, instance, instance + 1]
        valid_cond = chunk[chunk["instance_number"] == instance]["condition"].values[0]
        valid_rows = chunk[(chunk["instance_number"].isin(valid_ins)) & (chunk["condition"] == valid_cond)]
        irows = inference_rows(valid_rows, instance)
        erows = pd.concat([erows, irows], ignore_index = True)

print(len(sagittal_idf), len(erows))
itrain_labels = pd.concat([sagittal_idf, erows], ignore_index = True)
print(len(itrain_labels))
itrain_labels = itrain_labels.drop_duplicates(subset=['series_id', 'instance_number', 'level'])
itrain_labels = itrain_labels[train_labels.columns]
print(len(itrain_labels))

48692


/var/folders/j9/bcpct8w966j66c8j5tg5wd4c0000gn/T/ipykernel_39353/1008786243.py:21: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  erows = pd.concat([erows, irows], ignore_index = True)


48692 61899
110591
61941


In [ ]:
itrain_labels = itrain_labels.sort_values(['series_id', 'instance_number']).reset_index()
itrain_labels = itrain_labels[train_labels.columns]
print(itrain_labels[(itrain_labels["series_id"] == 1054713880) & (itrain_labels["instance_number"] == 11)])

      study_id   series_id series_description instance_number  \
15243  4003253  1054713880        Sagittal_T1              11   
15244  4003253  1054713880        Sagittal_T1              11   
15245  4003253  1054713880        Sagittal_T1              11   
15246  4003253  1054713880        Sagittal_T1              11   
15247  4003253  1054713880        Sagittal_T1              11   

                             condition  level           x           y  
15243  left_neural_foraminal_narrowing  l1_l2  114.374558   73.512367  
15244  left_neural_foraminal_narrowing  l4_l5  108.794275  146.762075  
15245  left_neural_foraminal_narrowing  l5_s1  114.975332  168.850095  
15246  left_neural_foraminal_narrowing  l2_l3  111.604240   99.236749  
15247  left_neural_foraminal_narrowing  l3_l4  109.595707  126.726297  


In [ ]:
"""Create appropriate dataframe containing the severity of the coordinate for the classification of each disc in each image"""
class_df = pd.merge(itrain_labels, train_series_descriptions, on = ["study_id", "series_id"])
class_df = class_df[class_df["series_description_x"] != "Axial_T2"].drop("series_description_y", axis = 1).rename(columns={"series_description_x": "series_description"})
class_df = class_df.dropna()

def rename_cell(col, val):
    return f"{col}_{val.lower().replace('/', '_')}"

def get_class(row):
    col_name = f"{row['condition']}_{row['level']}"  
    #print(col_name)
    if col_name in row.index: 
        return row[col_name]
    else:
        print(f"No column with name: {col_name}\n{row}"); return None

def class_to_number(cell, classes):
    return classes[cell] if classes[cell] is not None else np.nan

def process_nnunet_df(class_df, classes, st1_st2):
    train_copy = train[train.columns[:16]] #Drop subarticular columns. (Axial T2).
    n = 1 if st1_st2 == "st2" else 6
    m = 6 if st1_st2 == "st2" else 16
    cols = [train.columns[0]] + list(train.columns[n:m])
    train_copy = train[cols] #Drop correct condition columns, depending on sagt1 or t2. Including the study_id column.
    for col in train_copy.columns[1:]:
        train_copy[col] = train_copy[col].apply(lambda x: rename_cell(col, x))

    class_df = pd.merge(class_df, train_copy, on = "study_id")
    class_df['class_number'] = class_df.apply(get_class, axis=1)
    class_df = class_df.drop(class_df.columns[8:-1], axis = 1)
    class_df["class_number"] = class_df["class_number"].apply(lambda x: class_to_number(x, classes))
    if len(class_df.dropna()) != len(class_df): print("nans in the dataframe") 
    return class_df

class_df = class_df.drop(
    class_df[
        (class_df["series_description"] == "Sagittal_T1") & 
        (class_df["condition"].str.contains('canal'))
    ].index
)

class_df = class_df.drop(
    class_df[
        (class_df["series_description"] == "Sagittal_T2_STIR") & 
        (class_df["condition"].str.contains('narrowing'))
    ].index
)

st1_class_df = process_nnunet_df(class_df[class_df["series_description"] == "Sagittal_T1"], st1_classes, "st1")
st1_class_df = st1_class_df.dropna()
print("unqiue elements:", np.unique(class_df["series_description"].to_numpy()))
st2_class_df = process_nnunet_df(class_df[class_df["series_description"] == "Sagittal_T2_STIR"], st2_classes, "st2")
st2_class_df = st2_class_df.dropna()

if np.unique(st1_class_df["class_number"].to_numpy()).shape[0] != 30:
    print("Error in st1_class_df") 
if np.unique(st2_class_df["class_number"].to_numpy()).shape[0] != 15:
    print("Error in st2_class_df") 

/var/folders/j9/bcpct8w966j66c8j5tg5wd4c0000gn/T/ipykernel_39353/3544515472.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_copy[col] = train_copy[col].apply(lambda x: rename_cell(col, x))
/var/folders/j9/bcpct8w966j66c8j5tg5wd4c0000gn/T/ipykernel_39353/3544515472.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_copy[col] = train_copy[col].apply(lambda x: rename_cell(col, x))


unqiue elements: ['Sagittal_T1' 'Sagittal_T2_STIR']


In [ ]:
"""
Label the masks with the vertebrae class as well as the disc class. (From either st1_classes or st2_classes
General Process: -> Take the highest disc from the dataframe (l1_l2)
-> Find the center and sides of the vertebrae
-> Fill a rectangle below the disc, not overlapping the vertebrae below, representing the disc between the two vertebrae
-> Append to an array when completed
-> Repeat until l5_s1 is reached
-> Label s1
-> Compare the array of completed discs/vertebrae to the list of all discs
-> Move up to staring vertebrae, until all discs/vertebrae are completed
-> Label the remaining vertebrae as "other_disc": 2 #The name for these is wrong, however is it unimportant for now.
"""

def auto_label_mask(mask):
    labeled_mask, num_features = label(mask) 
    return labeled_mask
    
def correct_labels_for_overlap(mask, classes):
    #First change the auto-assigned background color(s) to 0.
    unique, counts = np.unique(mask, return_counts=True)
    sorted_indices = np.argsort(counts)[::-1] #Sort in descending order
    class_values = classes.values()
    for u in unique:
        if u == 0:
            continue
        if u in class_values:
            mask[mask == u] = u + 70 #70 ensures the overlapping pixel value is now completely unique.
    return mask

def auto_label_and_correct_mask(mask, classes):
    color_mask = auto_label_mask(mask)
    corrected_color_mask = correct_labels_for_overlap(color_mask, classes)
    return corrected_color_mask

def disc_to_vert_and_class(highest_disc, condition):
    n = 15 if condition == "spinal_canal_stenosis" else 0
    if highest_disc == "l1_l2": return "l1", (32 - n)
    elif highest_disc == "l2_l3": return "l2", (33 - n)
    elif highest_disc == "l3_l4": return "l3", (34 - n)
    elif highest_disc == "l4_l5": return "l4", (35 - n)
    elif highest_disc == "l5_s1": return "l5", (36 - n)


def align_verts_to_val(highest_vert, vert_val):
    if highest_vert == "l1": 
        return {"l1": vert_val + 0, "l2": vert_val + 1, "l3": vert_val + 2, "l4": vert_val + 3, "l5": vert_val + 4, "s1": vert_val + 5}
    elif highest_vert == "l2": 
        return {"l1": vert_val - 1, "l2": vert_val + 0, "l3": vert_val + 1, "l4": vert_val + 2, "l5": vert_val + 3, "s1": vert_val + 4}
    elif highest_vert == "l3": 
        return {"l1": vert_val - 2, "l2": vert_val - 1, "l3": vert_val + 0, "l4": vert_val + 1, "l5": vert_val + 2, "s1": vert_val + 3}
    elif highest_vert == "l4": 
        return {"l1": vert_val - 3, "l2": vert_val - 2, "l3": vert_val - 1, "l4": vert_val + 0, "l5": vert_val + 1, "s1": vert_val + 2}
    elif highest_vert == "l5": 
        return {"l1": vert_val - 4, "l2": vert_val - 3, "l3": vert_val - 2, "l4": vert_val - 1, "l5": vert_val + 0, "s1": vert_val + 1} 
    else:
        print(f"Vertebrae {highest_vert} invalid!")
    #There is no "s1" condition.
        
import matplotlib.patches as patches

def find_vert_with_coords(mask, x, y):
    central_x_est, central_y_est = int(x) - 11, int(y) - 8
    vert_val = mask[central_y_est, central_x_est]
    if vert_val == 0:
        offsets = [(dx, dy) for dx in range(-3, 4) for dy in range(-3, 4)]
        for dx, dy in offsets:
            temp_x_est, temp_y_est = central_x_est + dx, central_y_est + dy
            
            if 0 <= temp_y_est < mask.shape[0] and 0 <= temp_x_est < mask.shape[1]:
                vert_val = mask[temp_y_est, temp_x_est]
                if vert_val != 0:
                    return vert_val
        return None
    else:
        return vert_val

def locate_vertebrae(mask, highest_vert_str, highest_vert_class, x, y, all_vertebrae = ["l1", "l2", "l3", "l4", "l5", "s1"]):
    vertebrae_val = {v: 0 for v in all_vertebrae}
    #Find the highest vertebrae to act as a reference point within the image, as to what vertebrae are present and their class number.
    vert_val = find_vert_with_coords(mask, x, y)
    if vert_val == None:
        return None
    #Align this with order now.
    vert_val_dict = align_verts_to_val(highest_vert_str, vert_val)
    return vert_val_dict

def calculate_disc_bboxes(regions):
    #Unpack lx and lx+1 vert bboxes.
    bboxes = []
    for i in range(len(regions) - 1):
        top_u, left_u, bottom_u, right_u = regions[i] #Box above.
        top_l, left_l, bottom_l, right_l = regions[i + 1] #Box below.
        #Choose largest out of the x values, so nothing is missed.
        bbox = bottom_u - 3, min(left_u, left_l), top_l + 3, max(right_u + 5, right_l + 5) #Add some height to the bbox to fill dead space created otherwise.
        bboxes.append(bbox)
    return bboxes

def label_sag_mask(path, mask, chunk, classes, class_numbers_verts):    
    chunk = chunk.sort_values("level").reset_index() #Sort down l1_l2 -> l5_s1.
    x, y, level = chunk.x[0], chunk.y[0], chunk.level
    highest_disc = level[0]
    condition = chunk["condition"].values[0]
    highest_vert_str, highest_vert_class = disc_to_vert_and_class(highest_disc, condition)

    vert_val_dict = locate_vertebrae(mask, highest_vert_str, highest_vert_class, x, y) #Use these values to switch the values on the color mask to their respective class values.
    if vert_val_dict is None:
        #print("vert_val_dict is None.")
        return None
    for idx, vert_val in enumerate(vert_val_dict.values()):
        mask[mask == vert_val] = class_numbers_verts[idx] #Won't line up due to them being about 20-30 difference.
    #Assign all values but class_numbers and 0 to be a 1 (other_disc class).

    class_mask = mask.copy().astype(int)
    acceptable_vals = class_numbers_verts + [0]
    filter_mask = ~np.isin(class_mask, acceptable_vals) #Makes all other detected islands of pixels as a default value.
    class_mask[filter_mask] = 1 #~[0, 1, 32...37]

    no_other_mask = class_mask.copy()
    no_other_mask[no_other_mask == 1] = 0 #Make a copy of the classly labeled mask and remove the other_discs to simplify finding the disc regions between the vertebrae.
    no_other_mask = no_other_mask.astype(int)
    debug = np.unique(no_other_mask.copy())
    no_verts = len(np.unique(no_other_mask)) - 1
    regions = []
    for i in np.unique(no_other_mask):
        if i == 0:
            continue
        temp_mask = np.where(no_other_mask == i, i, 0)
        region = regionprops(temp_mask)
        regions.append(region[0].bbox)
    disc_bboxes = calculate_disc_bboxes(regions)

    #Using the class numbers and the bounding boxes, fill the class mask.
    class_numbers = chunk["class_number"].to_list() #Will be the values to assign the discs to (top down).
    def_discs = [2, 3, 4, 5, 6]
    if "right" in condition:
        def_discs = list(np.array(def_discs) + 15)

    all_discs = {"l1_l2": 0, "l2_l3": 0, "l3_l4": 0, "l4_l5": 0, "l5_s1": 0}
    for idx, disc in enumerate(all_discs.keys()):
        disc_info = chunk[chunk["level"] == disc]
        if disc_info.empty:
            all_discs[disc] = def_discs[idx]
        else:
            all_discs[disc] = class_numbers[0]
            class_numbers.remove(class_numbers[0])

    passive_mask = np.zeros_like(class_mask)
    class_numbers_padded = list(all_discs.values())
    for idx, bbox in enumerate(disc_bboxes):
        bottom, left, top, right = bbox
        passive_mask[bottom:top, left:right] = class_numbers_padded[idx]
    merged_mask = np.where(class_mask > 0, class_mask, passive_mask)                                #no_discs = no_vertebrae - 1.
    if 2 * no_verts - 3 != len(np.unique(merged_mask)) - 2: #Must follow the equation no_vertebrae + no_discs - classes[0, 1] == unique - classes[0, 1].
        return merged_mask, True
    else:
        return merged_mask, False


def save_mask_as_png(mask, fname, new_folder):
    assert mask.shape == (224, 224)
    fname = fname.replace("nnUNet_segments", new_folder).replace("npy", "png")
    save_folder = "/".join(fname.split('/')[:-1])
    os.makedirs(save_folder, exist_ok=True)
    mask = mask.astype(np.uint8)
    mask_pil = Image.fromarray(mask)
    mask_pil.save(fname)

In [ ]:
# Segement RSNA S1 Images.
fail_rates = {"Successful": 0, "Fail to load": 0, "Fail within loop": 0}
print(f"Labeling {len(st1_class_df.groupby(["series_id", "instance_number"]))} Sagittal T1 MRIs.")

for idx, chunk in st1_class_df.groupby(["series_id", "instance_number"]):
    segment_path = os.path.join("nnUNet_segments", str(chunk["study_id"].values[0]),
                                 str(chunk["series_id"].values[0]), f"{chunk["instance_number"].values[0]}.npy") 
    if os.path.isfile(segment_path):
        gray_mask = np.load(segment_path) #Is grayscale img.
    else:
        fail_rates["Fail to load"] = fail_rates["Fail to load"] + 1
        continue
    color_mask = auto_label_and_correct_mask(gray_mask, st1_classes)
    png_path = segment_path.replace("npy", "png").replace("nnUNet_segments", "train_png")
    sag_t1_mask, not_failed = label_sag_mask(path = png_path, mask = color_mask, chunk = chunk, classes = st1_classes, 
                                    class_numbers_verts = [32, 33, 34, 35, 36, 37])
    if not_failed:
        fail_rates["Successful"] = fail_rates["Successful"] + 1
        save_mask_as_png(sag_t1_mask, segment_path, 'nnUNet_segments_labeled')
    elif not not_failed:
        save_mask_as_png(sag_t1_mask, segment_path, 'nnUNet_segements_failed')
        fail_rates["Fail within loop"] = fail_rates["Fail within loop"] + 1
print(fail_rates)

Labeling 8282 Sagittal T1 MRIs.


TypeError: cannot unpack non-iterable NoneType object

In [ ]:
# Segment RSNA S2 images.
fail_rates = {"Successful": 0, "Fail to load": 0, "Fail within loop": 0}
print(f"Labeling {len(st2_class_df.groupby(["series_id", "instance_number"]))} Sagittal T2 MRIs.")
for idx, chunk in st2_class_df.groupby(["series_id", "instance_number"]):
    segment_path = os.path.join("nnUNet_segments", str(chunk["study_id"].values[0]),
                                str(chunk["series_id"].values[0]), f"{chunk["instance_number"].values[0]}.npy")
    if os.path.isfile(segment_path):
        gray_mask = np.load(segment_path)
    else:
        fail_rates["Fail to load"] = fail_rates["Fail to load"] + 1
        continue
    color_mask = auto_label_and_correct_mask(gray_mask, st2_classes)
    png_path = segment_path.replace("npy", "png").replace("nnUNet_segments", "train_png")
    sag_t2_mask = label_sag_mask(path = png_path, mask = color_mask, chunk = chunk, classes = st2_classes, 
                                    class_numbers_verts = [17, 18, 19, 20, 21, 22])
    if sag_t2_mask is not None:
        fail_rates["Successful"] = fail_rates["Successful"] + 1
        save_mask_as_png(sag_t2_mask, segment_path)
    else:
        fail_rates["Fail within loop"] = fail_rates["Fail within loop"] + 1
print(fail_rates)
print(f"Total number of Sagittal Samples: {len(glob(os.path.join("nnUNet_segments_labeled", "*/*/*png")))}")

Labeling 2521 Sagittal T2 MRIs.


TypeError: save_mask_as_png() missing 1 required positional argument: 'new_folder'

In [ ]:
def path_to_info(path):
    """Takes a path in the style that RSNA use."""
    sp = path.split('/')
    return sp[-3], sp[-2], sp[-1].split('.')[0] #Study_id, series_id, instance_number.

In [ ]:
# Stack masks and imgs on each other.
ST1_MASKS_DIR = "nnUNet_raw/Dataset001_ST1_Degeneration/labelsTr"
ST1_IMGS_DIR = "nnUNet_raw/Dataset001_ST1_Degeneration/imagesTr"
ST2_MASKS_DIR = "nnUNet_raw/Dataset002_ST2_Degeneration/labelsTr"
ST2_IMGS_DIR = "nnUNet_raw/Dataset002_ST2_Degeneration/imagesTr"
os.makedirs(ST1_MASKS_DIR, exist_ok=True)
os.makedirs(ST1_IMGS_DIR, exist_ok=True)
os.makedirs(ST2_MASKS_DIR, exist_ok=True)
os.makedirs(ST2_IMGS_DIR, exist_ok=True)

def stack_arrays(img1, img2):
    arr1 = np.array(img1).astype(np.uint8)
    arr2 = np.array(img2).astype(np.uint8)
    arrd = np.zeros_like(img1).astype(np.uint8)
    arr_stacked = np.stack([arr1, arr2, arrd], axis = 1)
    return arr_stacked.transpose(0, 2, 1)

def save_as_png(img, filepath):
    #img = (img - np.min(img)) / (np.max(img) - np.min(img)) * 255
    img = img.astype(np.uint8)
    img = Image.fromarray(img)
    img.save(filepath) 

def process_img(mask_path, img_path, masks_dir, imgs_dir, file_start, idx):
    mask = Image.open(mask_path).convert('L')
    maskk = np.array(mask).astype(np.uint8)
    img  = Image.open(img_path).convert('L')
    img = np.array(img).astype(np.uint8)
    new_mask_path = os.path.join(masks_dir, f"{file_start}_{f'{idx:04d}'}.png")
    new_img_path = os.path.join(imgs_dir, f"{file_start}_{f'{idx:04d}'}_0000.png")
    save_as_png(img, new_img_path)
    save_as_png(maskk, new_mask_path)

ST1_MASK_PATHS = []
ST2_MASK_PATHS = []
s1_sis = train_series_descriptions[train_series_descriptions["series_description"] == "Sagittal T1"]["series_id"].to_list()
s2_sis = train_series_descriptions[train_series_descriptions["series_description"] == "Sagittal T2/STIR"]["series_id"].to_list()
for path in glob(os.path.join("nnUNet_segments_labeled", "*/*/*png")):
    si = int(path.split('/')[-2])
    if si in s1_sis:
        ST1_MASK_PATHS.append(path)
    elif si in s2_sis:
        ST2_MASK_PATHS.append(path)

for i, mask_path in enumerate(ST1_MASK_PATHS):
    mask_path = ST1_MASK_PATHS
    img_path = mask_path.replace('nnUNet_segments_labeled', 'train_png')
    process_img(mask_path, img_path, ST1_MASKS_DIR, ST1_IMGS_DIR, "ST1", idx = i)


for i, mask_path in enumerate(len(ST2_MASK_PATHS)):
    mask_path = ST2_MASK_PATHS
    img_path = mask_path.replace('nnUNet_segments_labeled', 'train_png')
    process_img(mask_path, img_path, ST2_MASKS_DIR, ST2_IMGS_DIR, "ST2", idx = i)

AttributeError: 'list' object has no attribute 'replace'

In [ ]:
# Rename images and labels.
for i, path in enumerate(sorted(glob(os.path.join('nnUNet_raw/Dataset001_ST1_Degeneration/imagesTr', '*png')))):
    rename = f'nnUNet_raw/Dataset001_ST1_Degeneration/imagesTr/ST1_{i:04d}_0000.png'
    os.rename(path, rename)

for i, path in enumerate(sorted(glob(os.path.join('nnUNet_raw/Dataset001_ST1_Degeneration/labelsTr', '*png')))):
    rename = f'nnUNet_raw/Dataset001_ST1_Degeneration/labelsTr/ST1_{i:04d}.png'
    os.rename(path, rename)

In [ ]:
# Augment S1 to represent fair distribution of classes (2-31).

def get_dist(folder, dist):
    for path in glob.glob(os.path.join(folder, '*png')):
        mask = np.array(Image.open(path))
        unique = np.unique(mask)
        for n in unique:
            if n in dist.keys():
                dist[n] += 1
    return dist

dist = get_dist(folder=ST1_MASKS_DIR, dist={n: 0 for n in range(2, 32)})
dist

{2: 9104,
 3: 8179,
 4: 5583,
 5: 3359,
 6: 2489,
 7: 264,
 8: 1090,
 9: 3076,
 10: 4004,
 11: 973,
 12: 44,
 13: 44,
 14: 365,
 15: 529,
 16: 209,
 17: 8739,
 18: 7808,
 19: 5444,
 20: 3338,
 21: 2224,
 22: 353,
 23: 1245,
 24: 2894,
 25: 3818,
 26: 639,
 27: 11,
 28: 0,
 29: 121,
 30: 352,
 31: 187}

In [ ]:
# Clearly, augmentation is needed on 6-16, 21-31.
# Define the augmentation pipeline
augmentation_pipeline = A.Compose([
    A.HorizontalFlip(p=0.5),
    A.VerticalFlip(p=0.5),
    A.Rotate(limit=20, p=0.5),
    A.RandomBrightnessContrast(p=0.5),
    A.ShiftScaleRotate(shift_limit=0.1, scale_limit=0.1, rotate_limit=15, p=0.5),
    A.GaussianBlur(p=0.3),
    A.RandomGamma(p=0.3),
    A.ElasticTransform(p=0.3),
    A.CoarseDropout(max_holes=8, max_height=8, max_width=8, p=0.5)
])

augmented_numbers = list(range(6, 17)) + list(range(21, 32))
max_img_num = len(glob.glob(os.path.join(ST1_MASKS_DIR, '*png')))
for mask_path, img_path in zip(glob.glob(os.path.join(ST1_MASKS_DIR, '*png')), glob.glob(os.path.join(ST1_IMGS_DIR, '*png'))):
    mask, img = np.array(Image.open(mask_path)), np.array(Image.open(img_path))
    unique = np.unique(mask)
    if not any(np.isin(unique, augmented_numbers)):
        continue
    for i in range(10):
        # 10 Random albumentation which is also applied to the image.
        augmented = augmentation_pipeline(image=img, mask=mask)
        aug_img, aug_mask = augmented['image'], augmented['mask']

        aug_img_pil = Image.fromarray(aug_img)
        aug_mask_pil = Image.fromarray(aug_mask)

        aug_img_pil.save(os.path.join(ST1_IMGS_DIR, f'ST1_{max_img_num + i:4d}_0000.png'))
        aug_mask_pil.save(os.path.join(ST1_MASKS_DIR, f'ST1_{max_img_num + i:4d}.png'))

    max_img_num += 10

In [ ]:
# Check new dist.
get_dist(ST1_MASKS_DIR, {n: 0 for n in range(2, 32)})

{2: 9104,
 3: 8179,
 4: 5583,
 5: 3359,
 6: 2489,
 7: 264,
 8: 1090,
 9: 3076,
 10: 4004,
 11: 973,
 12: 44,
 13: 44,
 14: 365,
 15: 529,
 16: 209,
 17: 8739,
 18: 7808,
 19: 5444,
 20: 3338,
 21: 2224,
 22: 353,
 23: 1245,
 24: 2894,
 25: 3818,
 26: 639,
 27: 11,
 28: 0,
 29: 121,
 30: 352,
 31: 187}

In [ ]:
# There are quite alot of images now in the train dataset. Try taking a subset of the entire dataset with high quality segmenatations.
get_dist('nnUNet_raw/Dataset004_ST1/labelsTr', {n: 0 for n in range(2, 32)})

{2: 3,
 3: 3,
 4: 3,
 5: 2,
 6: 1,
 7: 0,
 8: 0,
 9: 0,
 10: 1,
 11: 1,
 12: 0,
 13: 0,
 14: 0,
 15: 0,
 16: 0,
 17: 44,
 18: 45,
 19: 40,
 20: 37,
 21: 0,
 22: 1,
 23: 0,
 24: 5,
 25: 7,
 26: 1,
 27: 0,
 28: 0,
 29: 0,
 30: 1,
 31: 0}

In [ ]:
# Change filenames of train imgs and masks then move to the appropriate folder.
def sd_to_bin(sd):
    if sd == "Sagittal T1": return "0000"
    if sd == "Sagittal T2/STIR": return "0001"
    if sd == "Axial T2": return "0002"

In [ ]:
# Preprocess test images and rename to format. -> seriesID_instanceNumber_000(0,1,2).png 
test_img_dir = "rsna-2024-lumbar-spine-degenerative-classification/test_images/44036939"
save_folders = ["nnUNet_raw/Dataset001_ST1_Degeneration/imagesTs", "axial_test_png", "nnUNet_raw/Dataset002_ST2_Degeneration/imagesTs"]
for idx, dicom_folder in enumerate(os.listdir(test_img_dir)):
        if idx == 0: 
            continue    
        print(dicom_folder, save_folders[idx - 1])

        sd = test_series_description[test_series_description["series_id"] == int(dicom_folder.split('/')[-1])]
        sd = sd["series_description"].values[0]
        bin_sd = sd_to_bin(sd)
        plane = "sagittal" if sd != "Axial T2" else "axial"
        dicom_data_3d = load_dicom_stack(os.path.join(test_img_dir, dicom_folder), plane = plane)
                
        if dicom_data_3d["array"].shape[0] == 0:
            print(f"Dicom data size 0: {dicom_data_3d["array"].shape}")
            continue
                
        save_folder = save_folders[idx - 1]
        os.makedirs(save_folder, exist_ok = True)
        for k in range(dicom_data_3d["array"].shape[0] - 1): 
            img1 = dicom_data_3d["array"][k]
            #print(type(img1))
            pixel_spacing = dicom_data_3d["pixel_spacing"]
            processed_img1 = preprocess_img(img1, pixel_spacing)
            #print(type(processed_img1))
            nn_img_path = os.path.join(save_folder, f"ST1_{k:04d}_0000.png") #{bin_to_sd}
            save_as_png(processed_img1, nn_img_path)
        

2828203845 nnUNet_raw/Dataset001_ST1_Degeneration/imagesTs


NameError: name 'sd_to_bin' is not defined

In [ ]:
st1_dataset = {
    "name": "Sagittal T1 Degeneration",
    "description": "Using Sagittal T1 images, identify areas of right/left neural foraminal narrowing",
    "reference": "",
    "licence": "",
    "release": "0.0",
    "tensorImageSize": "2D",

    "channel_names": { 
        "0": "mri", 
    }, 
    "labels": st1_classes,
    
    "numTraining": len(glob(os.path.join(UNET_ST1_DIR, "imagesTr", "*png"))), 
    "file_ending": ".png"
    }
with open(os.path.join(UNET_ST1_DIR, 'dataset.json'), 'w') as json_file:
    json.dump(st1_dataset, json_file, indent=4)

st2_dataset = {
    "name": "Sagittal T2/STIR Degeneration",
    "description": "Using Sagittal T2/STIR images, identify areas of spinal canal stenosis",
    "reference": "",
    "licence": "",
    "release": "0.0",
    "tensorImageSize": "2D",

    "channel_names": { 
        "0": "mri", 

    }, 
    "labels": st2_classes,
    
    "numTraining": len(glob(os.path.join(UNET_ST2_DIR, "imagesTr", "*png"))), 
    "file_ending": ".png"
    }
with open(os.path.join(UNET_ST2_DIR, 'dataset.json'), 'w') as json_file:
    json.dump(st2_dataset, json_file, indent=4)

In [ ]:
#Install nnUNet libraries.
%pip install nnunetv2 -q


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [5]:
import os
import glob
# Dataset002 trimmed is Dataset005_ST2
DS5_PATH = 'nnUNet_raw/Dataset005_ST2_Trimmed'
# Loop through DS5_PATH, deleting images not in the label folder
MASKS = glob.glob(os.path.join(DS5_PATH, 'labelsTr', '*png'))
IMAGES = [path.replace('labelsTr', 'imagesTr').replace('.png', '_0000.png') for path in MASKS]

In [9]:
# Copy images from Dataset002 to DS005.
import shutil
IMAGES_SRC = [path.replace('Dataset005_ST2_Trimmed', 'Dataset002_ST2_Degeneration') for path in IMAGES]
for path in IMAGES_SRC:
    path005 = path.replace('Dataset002_ST2_Degeneration', 'Dataset005_ST2_Trimmed')
    shutil.copy(path, path005)

In [21]:
# Rename the images and labels into 0001 - 00NM.
for i, (impath, labpath) in enumerate(zip(sorted(glob.glob(os.path.join(DS5_PATH, 'imagesTr', '*png'))), sorted(glob.glob(os.path.join(DS5_PATH, 'labelsTr', '*png')))), start=1):
    new_impath = impath.replace(impath[-13:], f'{i:04d}_0000.png')
    new_labpath = labpath.replace(labpath[-8:], f'{i:04d}.png')
    os.rename(impath, new_impath)
    os.rename(labpath, new_labpath)

In [28]:
# Then augment if contains numbers 
augment = list(range(7, 17))

import albumentations as A
import cv2
import numpy as np

augmentations = A.Compose([
    A.HorizontalFlip(p=1.0),
    A.Rotate(limit=45, p=1.0),                    # Rotate between -45 and 45 degrees
    A.ShiftScaleRotate(shift_limit=0.2, scale_limit=0.2, rotate_limit=0, p=1.0),  # Shift and scale
    A.CropAndPad(percent=(-0.2, 0.2), p=1.0),     # Random crop or padding
    A.RandomScale(scale_limit=0.2, p=1.0)         # Scale (stretch/shrink)
])

augment = list(range(7, 17))  # Specify values to look for in the mask

def augment_images(mask_path, img_path, current_index):
    # Load mask and image
    mask = cv2.imread(mask_path)
    image = cv2.imread(img_path)
    
    # Apply augmentations and save results
    for i in range(5):
        # Apply augmentation
        augmented = augmentations(image=image, mask=mask)
        augmented_image = augmented['image']
        augmented_mask = augmented['mask']

        new_impath = img_path.replace(img_path[-13:], f'{current_index:04d}_0000.png')
        new_labpath = mask_path.replace(mask_path[-8:], f'{current_index:04d}.png')
   
        cv2.imwrite(new_impath, augmented_image)
        cv2.imwrite(new_labpath, augmented_mask)

        current_index += 1
    

current_index = 1389 # New image to made.
for labpath, impath in zip(glob.glob(os.path.join(DS5_PATH, 'labelsTr', '*png')), glob.glob(os.path.join(DS5_PATH, 'imagesTr', '*png'))):
    mask = cv2.imread(path)
    if np.any(np.isin(mask, augment)):
        augment_images(labpath, impath, current_index)
        current_index += 5

/Users/jakecordery/Desktop/dissertation-york/.venv/lib/python3.12/site-packages/albumentations/__init__.py:24: UserWarning: A new version of Albumentations is available: 1.4.21 (you have 1.4.20). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()


In [34]:
import json
ds_5_json = {
    "name": "Sagittal T2/STIR Degeneration",
    "description": "Using Sagittal T2/STIR images, identify areas of spinal canal stenosis",
    "reference": "",
    "licence": "",
    "release": "0.0",
    "tensorImageSize": "2D",

    "channel_names": { 
        "0": "mri", 
    }, 
    "labels": st2_classes,
    
    "numTraining": len(glob.glob(os.path.join('nnUNet_raw', 'Dataset005_ST2_Trimmed', "imagesTr", "*png"))), 
    "file_ending": ".png"
    }
with open(os.path.join('nnUNet_raw/Dataset005_ST2_Trimmed', 'dataset.json'), 'w') as json_file:
    json.dump(ds_5_json, json_file, indent=4)

In [ ]:
import os
#Setup folder directories.
os.environ['nnUNet_preprocessed'] = "/Users/jakecordery/Desktop/Kaggle Competiton/lumbarSpineDegeneration/nnUNet_preprocessed"
os.environ['nnUNet_results'] = "/Users/jakecordery/Desktop/Kaggle Competiton/lumbarSpineDegeneration/nnUNet_results"
os.environ['nnUNet_raw'] = "/Users/jakecordery/Desktop/Kaggle Competiton/lumbarSpineDegeneration/nnUNet_raw"

In [ ]:
export nnUNet_preprocessed="/Users/jakecordery/Desktop/Kaggle Competiton/lumbarSpineDegeneration/nnUNet_preprocessed"
export nnUNet_results="/Users/jakecordery/Desktop/Kaggle Competiton/lumbarSpineDegeneration/nnUNet_results"
export nnUNet_raw="/Users/jakecordery/Desktop/Kaggle Competiton/lumbarSpineDegeneration/nnUNet_raw"

SyntaxError: invalid syntax (1624789386.py, line 1)

In [ ]:

#Plan and preprocess Sagittal T1 and T2 datasets.
!nnUNetv2_plan_and_preprocess -d 1 --verify_dataset_integrity
!nnUNetv2_plan_and_preprocess -d 2 --verify_dataset_integrity

In [ ]:
#Train and Predict Dataset 001.
!nnUNetv2_train 1 2d 5 -device mps 

In [ ]:

!nnUNetv2_predict -i "/Users/jakecordery/Desktop/Kaggle Competiton/lumbarSpineDegeneration/nnUNet_raw/Dataset001_ST1_Degeneration/imagesTs" -o "/Users/jakecordery/Desktop/Kaggle Competiton/lumbarSpineDegeneration/nnUNet_results/output" -d 001 -c 2d -f 0 --save_probabilities -device cpu

In [ ]:
#Train and Predict Dataset 002.
!nnUNetv2_train 2 2d 0 -device mps 
!nnUNetv2_predict -i "/Users/jakecordery/Desktop/Kaggle Competiton/lumbarSpineDegeneration/nnUNet_raw/Dataset002_ST2_Degeneration/imagesTs" -o "/Users/jakecordery/Desktop/Kaggle Competiton/lumbarSpineDegeneration/nnUNet_results/output" -d 002 -c 2d -f 0 --save_probabilities -device mps

## Evaluate Sagittal T1 Model Performance

In [ ]:
# Evaluate Sagittal T1 performance on a select test set of the data, covering a range of severities of conditions.
S1_PREDS_PATHS = glob(os.path.join('preds_16_11', '*png'))
S1_LABELS_PATH = glob(os.path.join('nnUNet_raw/Dataset004_ST1/labelsTs', '*png'))

In [ ]:
# Define colors for each class
st1_colors = {
    0: (0, 0, 0),       # background (black)
    1: (0, 0, 0), # other_disc (gray)
    # Left Neural Foraminal Narrowing - Normal (Aesthetic Green)
    2: (76, 175, 80),  # Light Green
    3: (76, 175, 80),  # Light Green
    4: (76, 175, 80),  # Light Green
    5: (76, 175, 80),  # Light Green
    6: (76, 175, 80),  # Light Green
    # Left Neural Foraminal Narrowing - Moderate (Aesthetic Yellow)
    7: (255, 235, 59), # Light Yellow
    8: (255, 235, 59), # Light Yellow
    9: (255, 235, 59), # Light Yellow
    10: (255, 235, 59),# Light Yellow
    11: (255, 235, 59),# Light Yellow
    # Left Neural Foraminal Narrowing - Severe (Aesthetic Red)
    12: (244, 67, 54), # Light Red
    13: (244, 67, 54), # Light Red
    14: (244, 67, 54), # Light Red
    15: (244, 67, 54), # Light Red
    16: (244, 67, 54), # Light Red
    # Right Neural Foraminal Narrowing - Normal (Aesthetic Green)
    17: (76, 175, 80), # Light Green
    18: (76, 175, 80), # Light Green
    19: (76, 175, 80), # Light Green
    20: (76, 175, 80), # Light Green
    21: (76, 175, 80), # Light Green
    # Right Neural Foraminal Narrowing - Moderate (Aesthetic Yellow)
    22: (255, 235, 59),# Light Yellow
    23: (255, 235, 59),# Light Yellow
    24: (255, 235, 59),# Light Yellow
    25: (255, 235, 59),# Light Yellow
    26: (255, 235, 59),# Light Yellow
    # Right Neural Foraminal Narrowing - Severe (Aesthetic Red)
    27: (244, 67, 54), # Light Red
    28: (244, 67, 54), # Light Red
    29: (244, 67, 54), # Light Red
    30: (244, 67, 54), # Light Red
    31: (244, 67, 54), # Light Red
    
    # Sagittal T1 Disc - Assigned a unique color range for each
    32: (128, 128, 128),  #
    33: (128, 128, 128),  # 
    34: (128, 128, 128),   # L3 - Darker Blue
    35: (128, 128, 128),   # L4 - Even Darker Blue
    36: (128, 128, 128),    # L5 - Very Dark Blue
    37: (128, 128, 128)    # S1 - Grayish-Black
}

def apply_color_mapping(array):
    # Create an empty color image array (224, 224, 3) for RGB channels
    color_image = np.zeros((array.shape[0], array.shape[1], 3), dtype=np.uint8)

    # Apply colors to each class ID in the array
    for class_id, color in st1_colors.items():
        mask = (array == class_id)
        color_image[mask] = color  # Assign the RGB color to the masked region
    return color_image

os.makedirs('colored_output', exist_ok=True)
for i, (pred, label) in enumerate(zip(S1_PREDS_PATHS, S1_LABELS_PATH), start=1):
    pred = cv2.imread(pred, cv2.IMREAD_GRAYSCALE)
    label = cv2.imread(label, cv2.IMREAD_GRAYSCALE)
    pretty_pred = apply_color_mapping(pred)
    pretty_label = apply_color_mapping(label)

    pred_path = os.path.join('colored_output', f'pred_{i}.png')
    label_path = os.path.join('colored_output', f'label_{i}.png')
    cv2.imwrite(pred_path, pretty_pred)
    cv2.imwrite(label_path, pretty_label)

## Axial nnU-Net setup

In [ ]:
ls, rs = "left_subarticular_stenosis", "right_subarticular_stenosis"
ax_classes = {
    "background": 0,

    f"{ls}_{l1}_{n}": 1, #Axial: left_subarticular_stenosis n/m/s.
    f"{ls}_{l2}_{n}": 2,
    f"{ls}_{l3}_{n}": 3,
    f"{ls}_{l4}_{n}": 4,
    f"{ls}_{l5}_{n}": 5,
    f"{ls}_{l1}_{m}": 6,
    f"{ls}_{l2}_{m}": 7,
    f"{ls}_{l3}_{m}": 8,
    f"{ls}_{l4}_{m}": 9,
    f"{ls}_{l5}_{m}": 10,
    f"{ls}_{l1}_{s}": 11,
    f"{ls}_{l2}_{s}": 12,
    f"{ls}_{l3}_{s}": 13,
    f"{ls}_{l4}_{s}": 14,
    f"{ls}_{l5}_{s}": 15,

    f"{rs}_{l1}_{n}": 16, #Axial: right_subarticular_stenosis n/m/s.
    f"{rs}_{l2}_{n}": 17,
    f"{rs}_{l3}_{n}": 18,
    f"{rs}_{l4}_{n}": 19,
    f"{rs}_{l5}_{n}": 20,
    f"{rs}_{l1}_{m}": 21,
    f"{rs}_{l2}_{m}": 22,
    f"{rs}_{l3}_{m}": 23,
    f"{rs}_{l4}_{m}": 24,
    f"{rs}_{l5}_{m}": 25,
    f"{rs}_{l1}_{s}": 26,
    f"{rs}_{l2}_{s}": 27,
    f"{rs}_{l3}_{s}": 28,
    f"{rs}_{l4}_{s}": 29,
    f"{rs}_{l5}_{s}": 30,
}

In [ ]:
"""Create class_df for the axial plane"""
def rename_cell(col, val):
    return f"{col}_{val.lower().replace('/', '_')}"

def get_class(row):
    col_name = f"{row['condition']}_{row['level']}"  
    if col_name in row.index: 
        return row[col_name]
    else:
        print(f"No column with name: {col_name}\n{row}"); return None

def class_to_number(cell, classes):
    return classes[cell] if classes[cell] is not None else np.nan

def aprocess_nnunet_df(class_df, classes):
    cols = list(train.columns)
    ax_train_cols = [cols[0]] + cols[16:]
    train_copy = train[ax_train_cols] #Drop all but subarticular columns.
    for col in train_copy.columns[1:]:
        train_copy[col] = train_copy[col].apply(lambda x: rename_cell(col, x))

    class_df = pd.merge(class_df, train_copy, on = "study_id")
    class_df['class_number'] = class_df.apply(get_class, axis=1)
    #class_df = class_df.drop(class_df.columns[8:-1], axis = 1)
    class_df["class_number"] = class_df["class_number"].apply(lambda x: class_to_number(x, classes))
    if len(class_df.dropna()) != len(class_df): print("nans in the dataframe") 
    return class_df

axtrain_labels = train_labels[(train_labels["series_description"] == "Axial T2") | (train_labels["series_description"] == "Axial_T2")]
class_df = pd.merge(axtrain_labels, train_series_descriptions, on = ["study_id", "series_id"])
class_df = class_df.dropna()
ax_class_df = aprocess_nnunet_df(class_df, ax_classes)
ax_class_df = ax_class_df.dropna()

/var/folders/j9/bcpct8w966j66c8j5tg5wd4c0000gn/T/ipykernel_39353/3334558908.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_copy[col] = train_copy[col].apply(lambda x: rename_cell(col, x))


In [ ]:
# Basic axial segmentations.
fail = {'success': 0,'none': 0}
#Use the RSNA coordinates of the degeneration and just create a 5x5 pixel box around it and feed into nnU-Net.
def get_coords_class_number(path):
    sd, si, ino = path_to_info(path)
    tdf = ax_class_df[(ax_class_df["series_id"] == int(si)) & (ax_class_df["instance_number"] == int(ino))]
    if tdf.empty:
        return None, None, None
    else:
        return tdf["x"].values[0], tdf["y"].values[0], tdf["class_number"].values[0]
    
def create_seg_mask(x, y, class_number):
    blank_mask = np.zeros((224, 224), dtype = np.int8)
    #Create a 'class_number box' to do a very simple segmentation of the degenerative area.
    blank_mask[(int(x) - 5):(int(x) + 5), (int(y) - 5):(int(y) + 5)] = class_number 
    return blank_mask.copy()

def create_segments_axial(img_paths, dataset003_path="nnUNet_raw/Dataset003_AX2_Degeneration", impath="imagesTr", labpath="labelsTr"):
    file_idx = 0
    os.makedirs(os.path.join(dataset003_path, impath), exist_ok = True)
    os.makedirs(os.path.join(dataset003_path, labpath), exist_ok = True)
    for path in img_paths:
        x, y, class_number = get_coords_class_number(path)
        if x is not None:
            fail['success'] += 1
            mask = create_seg_mask(x, y, class_number)
            nn_img_path = os.path.join(dataset003_path, impath, f"AX2_{f'{file_idx:04}'}_0000.png")
            nn_mask_path = os.path.join(dataset003_path, labpath, f"AX2_{f'{file_idx:04d}'}.png")
            shutil.copy(path, nn_img_path)
            save_as_png(mask, nn_mask_path)
            file_idx += 1
        else:
            fail['none'] += 1

create_segments_axial(AX_IMG_PATHS)
print(fail)

{'success': 13724, 'none': 66255}


In [ ]:
ax2_dataset = {
    "name": "Axial T2 Degeneration",
    "description": "",
    "reference": "",
    "licence": "",
    "release": "0.0",
    "tensorImageSize": "2D",

    "channel_names": { 
        "0": "mri", 

    }, 
    "labels": ax_classes,
    
    "numTraining": len(glob(os.path.join('nnUNet_raw/Dataset003_AX2_Degeneration/imagesTr', "*png"))), 
    "file_ending": ".png"
    }
with open(os.path.join('nnUNet_raw/Dataset003_AX2_Degeneration', 'dataset.json'), 'w') as json_file:
    json.dump(ax2_dataset, json_file, indent=4)

In [ ]:
len(glob(os.path.join('nnUNet_raw/Dataset003_AX2_Degeneration/imagesTr', "*png"))), 

(13724,)